# Analysis of Boston/SeattleAirbnb Data

This project investigates Airbnb Data from Boston and Seattle.  

## Main Questions of the Project:
 
1) Which factors correlate with a high rental price for an object? How well can they predict the rental price?  
2) Can the factors that work in Boston predict prices in Seattle? Can the factors that work in Seattle predict prices in Boston?  
3) How well do the review scores match the NLTK language processing scores?  

## This Notebook: First look and Data Cleaning  

In [147]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
import pyarrow
%matplotlib inline

# Load in the Data from the two .csv files
boston_calendar = pd.read_csv('./boston_calendar.csv')
boston_listings = pd.read_csv('./boston_listings.csv')
boston_reviews = pd.read_csv('./boston_reviews.csv')
seattle_calendar = pd.read_csv('./seattle_calendar.csv')
seattle_listings = pd.read_csv('./seattle_listings.csv')
seattle_reviews = pd.read_csv('./seattle_reviews.csv')

## Data exploration
# Calendar.csv
First off, we start by exploring the datasets. Start with the calendar data set

In [148]:
# Take a look at the head
print("Calendar head\n\n", boston_calendar.head(),"\n\n")
# Take a look at the data types
print("Calendar data types \n\n" , boston_calendar.dtypes)

Calendar head

    listing_id        date available price
0    12147973  2017-09-05         f   NaN
1    12147973  2017-09-04         f   NaN
2    12147973  2017-09-03         f   NaN
3    12147973  2017-09-02         f   NaN
4    12147973  2017-09-01         f   NaN 


Calendar data types 

 listing_id     int64
date          object
available     object
price         object
dtype: object


There are 4 columns, a listings_id, the date, available (true,false) and a price. Checking the data types shows that dates are encoded as strings and not as dates. Further, the prices are also encoded as strings. As a first step, I convert the dates to dates and the prices to numbers.

In [149]:
def calendar_cleaning(calendar):
    # 1. Convert the dates from strings to dates
    calendar.date=pd.to_datetime(calendar.date)

    # 2. Convert available into boolean
    calendar.available=calendar.available.replace(to_replace=['f','t'],value=[False,True])

    # 3. Convert the prices from strings to numbers.
    # 3.1. Remove dollar signs
    calendar.price=calendar.price.str.lstrip('$')
    # 3.2. Remove , delimiters
    calendar.price=calendar.price.str.replace(',','', regex=True)
    # 3.3. Convert to Numbers
    calendar.price=pd.to_numeric(calendar.price)
    return(calendar)

boston_calendar=calendar_cleaning(boston_calendar)
seattle_calendar=calendar_cleaning(seattle_calendar)

Save the cleaned calendar data for later use

In [151]:
boston_calendar.to_parquet('boston_calendar_cleaned',index=False)
seattle_calendar.to_parquet('seattle_calendar_cleaned',index=False)

# Listings.csv

In [152]:
# Take a look at the head
print("Listings head\n\n", boston_listings.head(),"\n\n")
# Take a look at the data types
print("Listings data types \n\n" , boston_listings.dtypes)

Listings head

          id                            listing_url       scrape_id  \
0  12147973  https://www.airbnb.com/rooms/12147973  20160906204935   
1   3075044   https://www.airbnb.com/rooms/3075044  20160906204935   
2      6976      https://www.airbnb.com/rooms/6976  20160906204935   
3   1436513   https://www.airbnb.com/rooms/1436513  20160906204935   
4   7651065   https://www.airbnb.com/rooms/7651065  20160906204935   

  last_scraped                                           name  \
0   2016-09-07                     Sunny Bungalow in the City   
1   2016-09-07              Charming room in pet friendly apt   
2   2016-09-07               Mexican Folk Art Haven in Boston   
3   2016-09-07  Spacious Sunny Bedroom Suite in Historic Home   
4   2016-09-07                            Come Home to Boston   

                                             summary  \
0  Cozy, sunny, family home.  Master bedroom high...   
1  Charming and quiet room in a second floor 1910...   
2  C

Listings includes a large number of columns. I will select a subset of columns that I believe to be relevant to address the questions. I decided to drop most text based columns, as they would require analysis using some sort of text processing to be useful. For example, trying to analyze the description of an appartment to infer the tone or something like that.

In [153]:
cols=['id','host_response_time','host_response_rate','host_acceptance_rate','host_is_superhost','host_has_profile_pic','host_identity_verified','neighbourhood_cleansed','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','price','security_deposit','cleaning_fee','guests_included','extra_people','number_of_reviews','first_review','last_review','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','instant_bookable','cancellation_policy','require_guest_profile_picture','require_guest_phone_verification','reviews_per_month'
]
boston_listings=boston_listings[cols]
seattle_listings=seattle_listings[cols]

Now I turn to addressing the data types of the listings_sub dataframe

In [154]:
print(listings_sub.dtypes)

id                                       int64
host_response_rate                     float64
host_acceptance_rate                   float64
accommodates                             int64
bathrooms                              float64
                                        ...   
bed_type_Pull-out Sofa                   uint8
bed_type_Real Bed                        uint8
cancellation_policy_moderate             uint8
cancellation_policy_strict               uint8
cancellation_policy_super_strict_30      uint8
Length: 69, dtype: object


### Comments about the data types:
1) host response rate is object, should be float  
2) host acceptance rate is object, should be float  
3) Host is superhost needs to be bool  
4) Host has profile pic needs to be bool  
5) Host identity verified needs to be bool  
6) Price is string needs to be float  
7) Security deposit is string, needs to be float  
8) Cleaning fee is string, needs to be float  
9) Extra people is string, needs to be float  
10) First review is object, needs to be date  
11) Last review is object, needs to be date  
12) Instant bookable is string, needs to be bool  
13) require guest profile picture is string, needs to be bool  
14) require guest phone verification is string, needs to be bool

In [155]:
def listings_cleaning(listings_sub):
    
    
    ##### Converting the data types as needed #####

    # 1) host response rate is object, should be float
    # 1.1) Remove % signs
    listings_sub.host_response_rate=listings_sub.host_response_rate.str.replace('%','', regex=True)
    listings_sub.host_response_rate=pd.to_numeric(listings_sub.host_response_rate)
    # 2) host acceptance rate is object, should be float
    listings_sub.host_acceptance_rate=listings_sub.host_acceptance_rate.str.replace('%','', regex=True)
    listings_sub.host_acceptance_rate=pd.to_numeric(listings_sub.host_acceptance_rate)
    # 3) Host is superhost needs to be bool
    listings_sub.host_is_superhost=listings_sub.host_is_superhost.replace(to_replace=['f','t'],value=[False,True])
    # 4) Host has profile pic needs to be bool 
    listings_sub.host_has_profile_pic=listings_sub.host_has_profile_pic.replace(to_replace=['f','t'],value=[False,True])
    # 5) Host identity verified needs to be bool
    listings_sub.host_identity_verified=listings_sub.host_identity_verified.replace(to_replace=['f','t'],value=[False,True])
    # 6) Price is string needs to be float 
    # 6.1) Remove dollar signs
    listings_sub.price=listings_sub.price.str.lstrip('$')
    # 6.2) Remove , delimiters
    listings_sub.price=listings_sub.price.str.replace(',','', regex=True)
    # 6.3) Convert to Numbers
    listings_sub.price=pd.to_numeric(listings_sub.price)
    # 7) Security deposit is string, needs to be float
    # 7.1) Remove dollar signs
    listings_sub.security_deposit=listings_sub.security_deposit.str.lstrip('$')
    # 7.2) Remove , delimiters
    listings_sub.security_deposit=listings_sub.security_deposit.str.replace(',','', regex=True)
    # 7.3) Convert to Numbers
    listings_sub.security_deposit=pd.to_numeric(listings_sub.security_deposit)
    # 8) Cleaning fee is string, needs to be float  
    # 8.1) Remove dollar signs
    listings_sub.cleaning_fee=listings_sub.cleaning_fee.str.lstrip('$')
    # 8.2) Remove , delimiters
    listings_sub.cleaning_fee=listings_sub.cleaning_fee.str.replace(',','', regex=True)
    # 8.3) Convert to Numbers
    listings_sub.cleaning_fee=pd.to_numeric(listings_sub.cleaning_fee)
    # 9) Extra people is string, needs to be float
    # 9.1) Remove dollar signs
    listings_sub.extra_people=listings_sub.extra_people.str.lstrip('$')
    # 9.2) Remove , delimiters
    listings_sub.extra_people=listings_sub.extra_people.str.replace(',','', regex=True)
    # 9.3) Convert to Numbers
    listings_sub.extra_people=pd.to_numeric(listings_sub.extra_people)
    # 10) First review is object, needs to be date
    listings_sub.first_review=pd.to_datetime(listings_sub.first_review)
    # 11) Last review is object, needs to be date  
    listings_sub.last_review=pd.to_datetime(listings_sub.last_review)
    # 12) Instant bookable is string, needs to be bool
    listings_sub.instant_bookable=listings_sub.instant_bookable.replace(to_replace=['f','t'],value=[False,True])
    # 13) require guest profile picture is string, needs to be bool 
    listings_sub.require_guest_profile_picture=listings_sub.require_guest_profile_picture.replace(to_replace=['f','t'],value=[False,True])
    # 14) require guest phone verification is string, needs to be bool
    listings_sub.require_guest_phone_verification=listings_sub.require_guest_phone_verification.replace(to_replace=['f','t'],value=[False,True])
    
    ##### Converting the data types as needed #####

    ##### Addressing NaN's #####
    # Next, I will address missing values in the dataset. First, focus on the numeric columns of the dataset.
    listings_num= listings_sub.select_dtypes(include=np.number).copy()    
    # 1) Host response rate/acceptance sets NaN to 0  
    # 2) Extra people/Security deposit/cleaning fee NaN's are set to 0  
    # 3) Review per month NaN's are set to 0  
    listings_num.loc[:,['host_response_rate','host_acceptance_rate','security_deposit','cleaning_fee','reviews_per_month']]=listings_num.loc[:,['host_response_rate','host_acceptance_rate','security_deposit','cleaning_fee','reviews_per_month']].fillna(0)

    # 4) Other NaN's are set to the average
    fill_mean = lambda col: col.fillna(col.mean())

    try:
        listings_num=listings_num.apply(fill_mean, axis=0)
    except:
        print('That broke...')
        
    # Select the categorial variables
    listings_cat= listings_sub.select_dtypes(include='object').copy()
    # Use the methods from the lessons to fill the NaN's in the categorical variables
    for var in listings_cat.columns:
        # for each cat add dummy var, drop original column
        listings_cat = pd.concat([listings_cat.drop(var, axis=1), pd.get_dummies(listings_cat[var], prefix=var, prefix_sep='_', drop_first=True)], axis=1)

    # merge the dataframes
    listings_use = pd.concat([listings_num,listings_cat],axis=1)

    ##### Addressing NaN's #####
    
    # return the cleaned dataframe
    return listings_use

boston_listings = listings_cleaning(boston_listings)
seattle_listings = listings_cleaning(seattle_listings)

Save the cleaned listings data for later use

In [156]:
# save the cleaned data frame for later use
boston_listings.to_parquet('boston_listings_cleaned',index=False)
seattle_listings.to_parquet('seattle_listings_cleaned',index=False)

# Reviews.csv

In [157]:
# Import nltk sentiment analysis
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [158]:
def review_analysis(reviews):
    # Drop reviews that do not include a comment that can be analyzed
    reviews.dropna(subset=['comments'],inplace=True)
    # Create data frame to store results
    result=pd.DataFrame(columns=['listing_id','neg','neu','pos','compound'])
    for i in range(0,reviews.shape[0]):
        try:
            # Run sentiment analysis
            analysis = sia.polarity_scores(reviews.iloc[i,5])
            analysis['listing_id']=reviews.iloc[i,0]
            # Store result
            result = result.append(analysis,ignore_index=True)
        except:
            print(reviews.comments[i])
    return result

boston_reviews=review_analysis(boston_reviews)
seattle_reviews=review_analysis(seattle_reviews)

In [159]:
# save the cleaned data frame for later us
boston_reviews.to_parquet('boston_reviews_cleaned',index=False)
seattle_reviews.to_parquet('seattle_reviews_cleaned',index=False)